In [ ]:
# from functools import cache
import numpy as np
import pandas as pd
import csv
from typing import Dict, List



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install pyspark

Importing the modules

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import count, desc , col, max
# from pyspark.ml.feature import  StringIndexer
# from pyspark.ml import Pipeline

Creating the spark session

In [ ]:
# spark = SparkSession.builder.appName("lastfm").getOrCreate()

Load dataset from recommender using Pyspark

In [ ]:
# file_path = '/content/drive/MyDrive/scruf-d_datasets/'
# df_ls = spark.read.format('csv')
# df_wj = spark.read.format('csv')
# df_fi = spark.read.format('csv')

SyntaxError: ignored

In [ ]:
class Database:
  """contains all the supplied lists and generated lists of recommendations, fairness attributes"""
  Test_List = './datasets/test.csv'
  Recommender_List = './datasets/recommender.csv'
  Allocation_History = ''
  Fairness_Attributes= ''


  def get_recommender_list(self):
        return pd.read_csv(self.Recommender_List)
    
  def get_allocation_history(self):
        return pd.read_csv(self.Allocation_History)

  def get_fairness_attributes(self):
        return pd.read_csv(self.Fairness_Attributes)

  def get_test_list(self):
        return pd.read_csv(self.Test_List)

db = Database()
td = db.get_test_list()

FileNotFoundError: ignored

In [ ]:
class RecommenderSystem:
  """calls librec to generate recommendation lists"""
  

In [ ]:
#abstraction for supplyting and creating fairness agents
from abc import ABC, abstractmethod
from typing import Dict, List


class FairnessAgent(ABC):
  @abstractmethod 
  def mscore(self, items: List[str], **kwargs): 
      """
      Returns fairness score 
      """
  def compatibility(self, **kwargs): 
      """
      Returns compatibility score
      """
  def fi_efficacy(self, p, pp): 
      """
      Returns the efficacy of fairness agents
      """

       

Fairness Agents

In [ ]:
class SimpleFairnessAgent:
  """These fairness agents are associated with specific fairness concerns, assesing different aspects of the items, users, and the recommendation list ls."""
  td = db.get_test_list()
  rc = db.get_recommender_list()
  fa = db.get_fairness_attributes()
  
  
  def __init__(self, name, preference_map: Dict[Any, int], list_lookup_horizon):
        self.name = name
        self._preference_map = preference_map
        self.list_lookup_horizon = list_lookup_horizon
        self.norm = sorted(self._preference_map.values(), reverse=True)
        if len(self.norm) < list_lookup_horizon:
            # Pad with zeroes
            self.norm.extend([0] * (list_lookup_horizon - len(self.norm)))
        self.norm = np.array(self.norm)

  def preference(self, **kwargs) -> Dict[str, float]:
        return self._preference_map # read from csv 

  def point_fairness(self, items: List[str], **kwargs) -> float:
        N = min(self.list_lookup_horizon, len(items))
        list_weights = 1 / np.log(2 + np.arange(len(items)))
        gt = np.array([self._preference_map.get(i, 0.0) for i in items])
        return np.sum(gt[:N] * list_weights[:N]) / np.sum(self.norm[:N] * list_weights[:N])
    
  def mscore(self, **kwargs):
      total_items = df_ls['item_count'].sum()
      score = (((df_ls.groupby("protected")["item_count"].sum())/ total_items) - ((df_ls.groupby("unprotected")["item_count"].sum())/ total_items))
      return score
  def compatibility(self, ):
    
      return 1
  def preference(self, **kwargs):

  # def fi_recommendation(self,wj_mscore,wj_compatiblity,ls_preference,item_key):

  # def generate_lf(self, **kwargs):

  # def fi_efficacy(self, p, pp):
      # efficacy = min(1 - ((self.p - self.pp)/self.p))

NameError: ignored

Allocation Mechanism

In [ ]:
class AllocationMechanism:
    def allocate1(self,agents: List[Agent],fairness_history: Dict[Agent, List[float]]) -> Dict[Agent, float]: # make subclasses for the different variations of allocations
        d = {a: 5.0 for a in agents}
        for a, fairness in fairness_history.items():
            if a in d:
                d[a] += 1.0 - np.sum(fairness[-1:])
        agent = np.random.choice(agents, size=1, p=[d[a] / sum(d.values()) for a in agents])
        return {agent[0]: 1.0}


Choice Mechanism

In [ ]:
class ChoiceMechanism:

  def choice1():
    def choose(self, allocation: Dict[Agent, float]) -> Agent:
        agents, probas = list(allocation.keys()), np.array(list(allocation.values()))
        probas = probas / np.sum(probas)
        chosen = np.random.choice(agents, size=1, p=probas)[0]
        return chosen

    def apply_agent(self,agent: Agent, predicted_score: Dict[str, float]) -> Dict[str, float]:
        pref = agent.preference()
        scores = {i: predicted_score.get(i, 0.0) + pref.get(i, 0.0) for i in predicted_score}
        return scores

EXPERIMENT

EVALUATIONS

VISUALIZATIONS